In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd 
import requests as req
import dateutil as pdate
import seffaflik
from seffaflik.elektrik.piyasalar import gop
from seffaflik.elektrik import yekdem
from seffaflik.elektrik import tuketim
from openpyxl import load_workbook
import datetime 
import time
from __future__ import unicode_literals

x = datetime.datetime.now() 
today = datetime.date.today() 
tomorrow = today + datetime.timedelta(days = 1)  
tomorrow = str(tomorrow)
lastweekd= today - datetime.timedelta(days=5)
lastweekd= str(lastweekd)

arz_talep = gop.arz_talep_egrisi(tarih=tomorrow)

col_name="Fiyat"
first_col = arz_talep.pop(col_name)
arz_talep.insert(0, col_name, first_col)

grouped = arz_talep.groupby(arz_talep.Saat)

df_list = ['df'+str(i) for i in range(0,24)]

df_list[0] = grouped.get_group(0)
df_list[0] = df_list[0].drop(columns=['Saat'])


for j in range (1,24): 
        df_list[j] = grouped.get_group(j)  
        df_list[j]= df_list[j].drop(columns=['Saat'])
        df_list[j]= df_list[j].reset_index(drop=True)
        df_list[0]= df_list[0].reset_index(drop=True )
        df_list[0]= pd.concat([df_list[0],df_list[j]], axis=1)
df_list[0]["Arz"]= df_list[0]["Arz"] * -1

path = "Arz_Talep.xlsx"

writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

df_list[0].to_excel(writer,sheet_name = tomorrow)

res_tahmin = yekdem.res_uretim_tahmini()
kaldir = ["Q5","Q25","Q75","Q95"]
for i in kaldir :
    res_tahmin.pop(i)

res_tahmin.to_excel(writer, sheet_name = 'RES_TAHMIN')
blok_tek= gop.blok_miktari(baslangic_tarihi=tomorrow,bitis_tarihi=tomorrow)
sutunkaldir= ["Talep Blok Teklif Miktarı","Arz Blok Teklif Miktarı"]
for k in sutunkaldir: 
    blok_tek.pop(k)
blok_tek.to_excel(writer,sheet_name ="BLOK ALIS SATIS")
gt=tuketim.gerceklesen(baslangic_tarihi=lastweekd,bitis_tarihi=str(today))



gt.to_excel(writer,sheet_name = "GECMIS TUKETIM")
writer.save()
